In [1]:
import requests
from bs4 import BeautifulSoup
import urllib.parse
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
url = 'https://www.aljazeera.net/ebusiness/'

response = requests.get(url)

response = response.content

In [3]:
soup = BeautifulSoup(response, 'html.parser')

### Get all articles in the website

In [4]:
all_articles = soup.find_all('article')

### Get only articles within a specific section

In [5]:
section = soup.find('section', id='news-feed-container')
section

<section aria-label="تدفق المحتوى" id="news-feed-container" tabindex="-1"><h2 class="screen-reader-text">تدفق المحتوى</h2><article class="gc u-clickable-card gc--type-post gc--list gc--with-image"><div class="gc__image-placeholder"></div><div class="gc__content"><div class="gc__header-wrap"><div class="gc__header-meta u-clickable-card__exclude"><div class="post-label"><span class="post-label__text">مراسلو الجزيرة نت</span></div></div><h3 class="gc__title"><a class="u-clickable-card__link" href="/ebusiness/2024/9/3/%d9%83%d9%8a%d9%81-%d8%aa%d8%aa%d8%b9%d8%a7%d9%85%d9%84-%d8%a5%d9%8a%d8%b1%d8%a7%d9%86-%d9%85%d8%b9-%d8%aa%d9%87%d8%b1%d9%8a%d8%a8-%d8%a7%d9%84%d9%88%d9%82%d9%88%d8%af-%d9%81%d9%8a-2"><span>كيف تتعامل إيران مع تهريب الوقود في الخليج العربي؟</span></a></h3></div><div class="gc__body-wrap"><div class="gc__excerpt"><p>تعاني إيران الغنية بمعادن الطاقة من تهريب الوقود إلى خارج حدودها. وتضبط قواتها البحرية بشكل مستمر عمليات تهريب الوقود، لا سيما الديزل، في مياه الخليج العربي التي ت

In [6]:
articles = section.find_all('article',class_='gc u-clickable-card gc--type-post gc--list gc--with-image')

In [7]:
def extract_article_details(article):
    img = article.find('img')
    
    title = ''
    url=''
    title_tag = article.find('h3', class_='gc__title')
    if title_tag:
        span_tag = title_tag.find('span')
        if span_tag:
            title = span_tag.text
        a_tag = title_tag.find('a')
        if a_tag and 'href' in a_tag.attrs:
            url = a_tag['href']
            url = urllib.parse.urljoin("https://www.aljazeera.net/", url)  # Optional: If the URL is relative, make it absolute
            url = urllib.parse.unquote(url)  
    
    descr = ''
    descr_tag = article.find('div', class_='gc__excerpt')
    if descr_tag:
        p_tag = descr_tag.find('p')
        if p_tag:
            descr = p_tag.text
    
    date = ''
    footer_tag = article.find('footer')
    if footer_tag:
        span_tags = footer_tag.find_all('span')
        if len(span_tags) > 1:
            date = span_tags[1].text
    

    return img, title, descr, date, url



In [44]:
for article in all_articles:
    extract_article_details(article)

URL: https://www.aljazeera.net/ebusiness/2024/9/2/بوتاش-التركية-وشل-البريطانية-توقعان
URL: https://www.aljazeera.net/ebusiness/2024/9/2/الدولار-يلامس-أعلى-مستوى-له-قبل-صدور
URL: https://www.aljazeera.net/ebusiness/2024/9/2/محللون-نتنياهو-يخشى-الاحتجاجات-وقد
URL: https://www.aljazeera.net/ebusiness/2024/9/2/النفط-يواصل-خسائره-واحتمالات-زيادة
URL: https://www.aljazeera.net/politics/2024/9/2/صلتك-القطرية-توفر-أكثر-من-371-فرصة-عمل
URL: https://www.aljazeera.net/ebusiness/2024/9/2/اللايقين-يستنزف-إسرائيل
URL: https://www.aljazeera.net/tech/2024/9/1/مستثمرون-يتوقعون-صعودها-قيمة-أوبن
URL: https://www.aljazeera.net/ebusiness/2024/9/1/تفاقم-أزمة-الكهرباء-في-إيران-إليك-حجم
URL: https://www.aljazeera.net/ebusiness/2024/9/1/قطر-للطاقة-تضاعف-إنتجها-من-اليوريا
URL: https://www.aljazeera.net/ebusiness/2024/9/1/العراق-سيعرض-10-مناطق-للتنقيب-عن-الغاز
URL: https://www.aljazeera.net/ebusiness/2024/9/1/قطاع-الطاقة-الشمسية-في-الصين-يواجه
URL: https://www.aljazeera.net/ebusiness/2024/9/1/3-حقول-نفط-ليبية-تت

### Function to get a specific number of articles

In [8]:
def get_all_urls(base_url, max_articles=30):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    
    driver.get(base_url)
    
    all_articles = []

    while len(all_articles) < max_articles:
        try:
            # Wait for the "Show More" button to be clickable
            load_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button.show-more-button[data-testid='show-more-button']"))
            )
            
            # Scroll the button into view
            driver.execute_script("arguments[0].scrollIntoView(true);", load_more_button)
            time.sleep(1)  # Give it a moment to scroll
            
            # Click the "Show More" button
            load_more_button.click()
            time.sleep(2)  # Wait for new content to load

            # Parse the page after each click
            soup = BeautifulSoup(driver.page_source, "html.parser")
            articles = soup.find_all('article')
            
            # Process each article
            for article in articles:
                if article not in all_articles:
                    all_articles.append(article)

            # Stop if we have collected enough articles
            if len(all_articles) >= max_articles:
                break

        except Exception as e:
            print("Exception occurred while clicking 'Show More':", e)
            break

    # Close the browser
    driver.quit()

    # Extract details from the collected articles
    article_details = []
    for article in all_articles[:max_articles]:
        img, title, descr, date, url = extract_article_details(article)
        article_details.append({
            'title': title,
            'description': descr,
            'date': date,
            'url': url
        })

    return article_details

In [9]:
# Example usage
base_url = "https://www.aljazeera.net/ebusiness/"
all_articles = get_all_urls(base_url, max_articles=30)

for article in all_articles:
    print(f"Title: {article['title']}")
    print(f"Description: {article['description']}")
    print(f"Date: {article['date']}")
    print(f"URL: {article['url']}")
    print("---")

Title: هل تسعى تركيا إلى "بريكس" لتبتعد عن الغرب؟
Description: تسعى تركيا لتوسيع تحالفاتها العالمية وتطوير علاقاتها مع القوى الصاعدة عبر الانضمام إلى مجموعة “بريكس”، وهي مجموعة اقتصادية تضم بعضًا من أكبر الاقتصادات الناشئة في العالم. 
Date: 3/9/2024
URL: https://www.aljazeera.net/ebusiness/2024/9/3/هل-تسعى-تركيا-إلى-بريكس-لتبتعد-عن
---
Title: إجراءات البنك المركزي العراقي الجديدة بين التحديات المحلية والعقوبات الأميركية
Description: أعلن البنك المركزي العراقي نتائج زيارة وفده الذي ترأسه المحافظ علي العلاق إلى واشنطن، ولقائه مع ممثلي الخزانة الأميركية والبنك الاحتياطي الفدرالي، بالإضافة إلى لقاءات مع مجموعة من الشركات والمصارف.
Date: 3/9/2024
URL: https://www.aljazeera.net/ebusiness/2024/9/3/إجراءات-البنك-المركزي-العراقي
---
Title: ما الفوائد التي ستجنيها الجزائر بعد انضمامها إلى بنك البريكس؟
Description: تمت الموافقة رسميا على انضمام الجزائر الى بنك البريكس، في ختام الاجتماع السنوي لمجلس محافظي البنك الجديد للتنمية، الذي انعقد في كيب تاون بجنوب أفريقيا وهي خطوة كبيرة في مسار الاندماج ا

In [11]:
all_articles

[{'title': 'هل تسعى تركيا إلى "بريكس" لتبتعد عن الغرب؟',
  'description': 'تسعى تركيا لتوسيع تحالفاتها العالمية وتطوير علاقاتها مع القوى الصاعدة عبر الانضمام إلى مجموعة “بريكس”، وهي مجموعة اقتصادية تضم بعضًا من أكبر الاقتصادات الناشئة في العالم.\xa0',
  'date': '3/9/2024',
  'url': 'https://www.aljazeera.net/ebusiness/2024/9/3/هل-تسعى-تركيا-إلى-بريكس-لتبتعد-عن'},
 {'title': 'إجراءات البنك المركزي العراقي الجديدة بين التحديات المحلية والعقوبات الأميركية',
  'description': 'أعلن البنك المركزي العراقي نتائج زيارة وفده الذي ترأسه المحافظ علي العلاق إلى واشنطن، ولقائه مع ممثلي الخزانة الأميركية والبنك الاحتياطي الفدرالي، بالإضافة إلى لقاءات مع مجموعة من الشركات والمصارف.',
  'date': '3/9/2024',
  'url': 'https://www.aljazeera.net/ebusiness/2024/9/3/إجراءات-البنك-المركزي-العراقي'},
 {'title': 'ما الفوائد التي ستجنيها الجزائر بعد انضمامها إلى بنك البريكس؟',
  'description': 'تمت الموافقة رسميا على انضمام الجزائر الى بنك البريكس، في ختام الاجتماع السنوي لمجلس محافظي البنك الجديد للتنمية، الذي 

In [13]:
article_url = all_articles[2]['url']
article_url


'https://www.aljazeera.net/ebusiness/2024/9/2/ما-الفوائد-التي-ستجنيها-الجزائر-بعد'

### Scraping one article

In [14]:
response = requests.get(article_url)

soup = BeautifulSoup(response.content, 'html.parser')

main_content = soup.find('main', id='main-content-area')

main_content

<main aria-label="مجال المحتوى الرئيسي" id="main-content-area" tabindex="-1"><header class="article-header"><div class="breadcrumbs"><div class="topics"><a href="/ebusiness/">اقتصاد</a>|<a href="/where/arab/">عربي</a></div></div><h1>ما الفوائد التي ستجنيها الجزائر بعد انضمامها إلى بنك البريكس؟</h1></header><figure class="article-featured-image"><div class="responsive-image"><img alt="الانضمام إلى بنك بريكس خطوة كبيرة في مسار الاندماج في النظام المالي العالمية وزارة المالية الجزائرية الصفحة الرسمية" fetchpriority="high" loading="eager" sizes="(max-width: 120px) 120px, (max-width: 270px) 270px, (max-width: 375px) 375px, (max-width: 570px) 570px, (max-width: 770px) 770px, 770px" src="/wp-content/uploads/2024/09/غع7-1725282505.jpg?resize=770%2C513&amp;quality=80" srcset="/wp-content/uploads/2024/09/غع7-1725282505.jpg?resize=120%2C80&amp;quality=80 120w, /wp-content/uploads/2024/09/غع7-1725282505.jpg?resize=270%2C180&amp;quality=80 270w, /wp-content/uploads/2024/09/غع7-1725282505.jpg?resize

In [19]:
base_url = 'https://www.aljazeera.net'


header = main_content.find('header')
topics = header.find('div', class_='topics')
links = topics.find_all('a')

topics_and_urls = {}
for link in links:
    topic = link.get_text()
    relative_url = link.get('href')
    absolute_url = base_url + relative_url
    topics_and_urls[topic] = absolute_url
 
topics_and_urls


{'اقتصاد': 'https://www.aljazeera.net/ebusiness/',
 'عربي': 'https://www.aljazeera.net/where/arab/'}

In [20]:
title = header.find('h1').get_text()
title

'ما الفوائد التي ستجنيها الجزائر بعد انضمامها إلى بنك البريكس؟'

In [22]:
figure = main_content.find('figure')

img = figure.find('img')
img_src = img.get('src')
img_url = base_url + img_src

figcaption = figure.find('figcaption').get_text()

print("Image URL:", img_url)
print("Figure Caption:", figcaption)

Image URL: https://www.aljazeera.net/wp-content/uploads/2024/09/غع7-1725282505.jpg?resize=770%2C513&quality=80
Figure Caption: الانضمام إلى بنك بريكس خطوة كبيرة في مسار الاندماج في النظام المالي العالمي (وزارة المالية الجزائرية)


In [38]:
author_info = main_content.find('a', class_='author-link')

author_link = author_info.get('href')
author_link = base_url + author_link
author_link = urllib.parse.unquote(author_link)  
author_link

author_name = author_info.get_text()
author_name

'إلهام محمد'

In [39]:
publish_date = main_content.find('span', class_='article-dates__published').get_text()
publish_date


'2/9/2024'

In [43]:
article_content = main_content.find('div', class_='wysiwyg wysiwyg--all-content css-1vkfgk0')

article_content = article_content.get_text(separator='\n', strip=True)
article_content

'الجزائر-\nأعلنت وزارة المالية الجزائرية أنه تمت الموافقة رسميا على انضمام الجزائر الى بنك\nالبريكس\nوذلك في ختام الاجتماع السنوي التاسع لمجلس محافظي البنك الجديد للتنمية "إن دي بي" (NDB)، الذي انعقد السبت 31 أغسطس/آب في كيب تاون بجنوب أفريقيا.\nواعتبرت وزارة المالية الجزائرية أن انضمام\nالجزائر\nإلى هذه المؤسسة التنموية الهامة، التي تُعتبر الذراع المالية لمجموعة بريكس، خطوة كبيرة في مسار الاندماج في النظام المالي العالمي، مما يجعل الجزائر الدولة التاسعة التي تنضم إلى عضوية البنك الجديد للتنمية.\nوناقش وزير المالية الجزائري لعزيز فايد في أبريل/نيسان الماضي بواشنطن مع ديلما فانا روسيف، رئيسة بنك التنمية الجديد تطور عملية انضمام الجزائر إلى هذه المؤسسة المالية الدولية، كما ناقش الطرفان إمكانات التعاون مستقبلا ما بين البنك والجزائر في تمويل المشاريع التنموية الإستراتيجية والهيكلية للاقتصاد الجزائري.\nوكان الرئيس الجزائري\nعبد المجيد تبون\nقد كشف في ختام زيارته للصين العام الماضي أن الجزائر طلبت رسميا الانضمام الى البنك التابع لمجموعة "بريكس" كعضو، وبمساهمة أولى قدرها 1.5 مليار دولار.\nالج